# Predicting whether a player is likely to contribute a large amount of data #

In [ ]:
import pandas as pd
import altair as alt
import numpy as np

from sklearn import set_config
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import GridSearchCV, cross_validate, train_test_split
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
from sklearn.compose import make_column_transformer
from sklearn.metrics.pairwise import euclidean_distances
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder, StandardScaler
from sklearn.metrics import mean_squared_error
from sklearn.metrics import recall_score, precision_score
from sklearn.compose import make_column_selector

# Output dataframes instead of arrays
set_config(transform_output="pandas")

## Introduction ##

PLAICraft is a data collection project that gathers gameplay data from Minecraft players. It is run by the UBC’s Pacific Laboratory for Artificial Intelligence (PLAI), whose goal is to advance artificial intelligence. This project specifically is focused on the creation of an AI that can understand and learn from its environment (Minecraft) called an embodied AI. The Project relies on data of players' speech and key presses. In order to help with data collection the team is interested in targeted recruiting for demographics that contribute the most data. To achieve this, PLAI has collaborated with students from DSCI 100 to identify the "kinds" of players most likely to provide a significant amount of data.

The provided dataset from `players.csv` has 9 variables: `experience`, `subscribe`, `played_hours`, `name` ,`gender`, `age`, `individualId`, and  `organizationName`. Among these, the variables `name` ,`gender`, `individualId`, and  `organizationName` are not related to any measured property relevant to this analysis and will not be considered. To answer what demographic contributes the most data, `played_ hours` will be our target variable. To help with downstream classification analysis players with more than 2 hours of playtime are categorized as "high", while those with less than 2 hours are categorized as "low” in the new column `play_time`. Although 2 hours is not a significant playtime, the overall play hours in the dataset are low; therefore, this threshold was set to create proportionate categories. Players who have high `play_hours` are the "kinds" of players who are likely to contribute a large amount of data. A trained K-Nearest Neighbor(KNN) will be used to classy whether or not someone will have high play time based on their `experience`, `subscribe`, `played_hours`, `name` , `age`. In addition a KNN regression model will be used to see if theres any trends with age and play hours.

## Methods & Results ##

In [55]:
player_data = pd.read_csv("data/players.csv")
player_data

,experience,subscribe,hashedEmail,played_hours,name,gender,age,individualId,organizationName
0,Pro,True,f6daba428a5e19a3d47574858c13550499be23603422e6...,30.3,Morgan,Male,9,NaN,NaN
1,Veteran,True,f3c813577c458ba0dfef80996f8f32c93b6e8af1fa9397...,3.8,Christian,Male,17,NaN,NaN
2,Veteran,False,b674dd7ee0d24096d1c019615ce4d12b20fcbff12d79d3...,0.0,Blake,Male,17,NaN,NaN
3,Amateur,True,23fe711e0e3b77f1da7aa221ab1192afe21648d47d2b4f...,0.7,Flora,Female,21,NaN,NaN
4,Regular,True,7dc01f10bf20671ecfccdac23812b1b415acd42c2147cb...,0.1,Kylie,Male,21,NaN,NaN
...,...,...,...,...,...,...,...,...,...
191,Amateur,True,b6e9e593b9ec51c5e335457341c324c34a2239531e1890...,0.0,Bailey,Female,17,NaN,NaN
192,Veteran,False,71453e425f07d10da4fa2b349c83e73ccdf0fb3312f778...,0.3,Pascal,Male,22,NaN,NaN
193,Amateur,False,d572f391d452b76ea2d7e5e53a3d38bfd7499c7399db29...,0.0,Dylan,Prefer not to say,17,NaN,NaN
194,Amateur,False,f19e136ddde68f365afc860c725ccff54307dedd13968e...,2.3,Harlow,Male,17,NaN,NaN


In [ ]:
# create a new column to show the classification of playtime
player_data["play_time"] = ['high' if played_hours >= 2 else 'low' for played_hours in player_data['played_hours']]
player_data

,experience,subscribe,hashedEmail,played_hours,name,gender,age,individualId,organizationName,play_time
0,Pro,True,f6daba428a5e19a3d47574858c13550499be23603422e6...,30.3,Morgan,Male,9,NaN,NaN,high
1,Veteran,True,f3c813577c458ba0dfef80996f8f32c93b6e8af1fa9397...,3.8,Christian,Male,17,NaN,NaN,high
2,Veteran,False,b674dd7ee0d24096d1c019615ce4d12b20fcbff12d79d3...,0.0,Blake,Male,17,NaN,NaN,low
3,Amateur,True,23fe711e0e3b77f1da7aa221ab1192afe21648d47d2b4f...,0.7,Flora,Female,21,NaN,NaN,low
4,Regular,True,7dc01f10bf20671ecfccdac23812b1b415acd42c2147cb...,0.1,Kylie,Male,21,NaN,NaN,low
...,...,...,...,...,...,...,...,...,...,...
191,Amateur,True,b6e9e593b9ec51c5e335457341c324c34a2239531e1890...,0.0,Bailey,Female,17,NaN,NaN,low
192,Veteran,False,71453e425f07d10da4fa2b349c83e73ccdf0fb3312f778...,0.3,Pascal,Male,22,NaN,NaN,low
193,Amateur,False,d572f391d452b76ea2d7e5e53a3d38bfd7499c7399db29...,0.0,Dylan,Prefer not to say,17,NaN,NaN,low
194,Amateur,False,f19e136ddde68f365afc860c725ccff54307dedd13968e...,2.3,Harlow,Male,17,NaN,NaN,high


In [89]:
player_data.info()
player_data["play_time"].value_counts()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 196 entries, 0 to 195
Data columns (total 10 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   experience        196 non-null    object 
 1   subscribe         196 non-null    bool   
 2   hashedEmail       196 non-null    object 
 3   played_hours      196 non-null    float64
 4   name              196 non-null    object 
 5   gender            196 non-null    object 
 6   age               196 non-null    int64  
 7   individualId      0 non-null      float64
 8   organizationName  0 non-null      float64
 9   play_time         196 non-null    object 
dtypes: bool(1), float64(3), int64(1), object(5)
memory usage: 14.1+ KB


play_time
low     170
high     26
Name: count, dtype: int64

In [ ]:
plot = alt.Chart(player_data).mark_circle().encode(
    x=alt.X("age")
        .title(["age"]),
    y=alt.Y("gender")
        .title(["gender"]),
    color=alt.Color("play_time")
        .title("play time")
).configure_axis(titleFontSize=19).properties(height=500,width=400)
plot

<<<<<<< local


alt.Chart(...)

alt.Chart(...)

>>>>>>> remote


### Age ###

A preliminary graph initially plotted to observe any outliers and help guide analysis. There were 6 data points that were very far from the rest: 4 data points with over 100 hr playtime and 2 data points with age over 60. Overall there did not seem to be any observable trends however for a more informative analysis K-Nearest Neighbors (KNN) regression was employed.The dataset (`filtered_player_data`) was split into training (`player_training`) and testing (`player_testing`) sets. The model was trained using the training data. First a column transformer was applied for preprocessing, and a model pipeline was created for KNNregression. Using 5-fold cross-validation measuring root mean square prediction error (RMSPE), the optimal K was selected from a parameter grid ranging from 1 to 110. K was determined to be 70, yielding a RMSPE value of 6.95 hours.To evaluate the model's accuracy on unseen data, the RMSPE was calculated on the test set, resulting in an RMSPE of 6.96 hours.Visualization was performed by plotting all player data points along with a prediction line generated by the KNNregression model. The prediction line appeared relatively linear, with a slight indentation observed around the 15-20 range.



#### Preliminary Graph ####

In [ ]:
#Plot preliminary graph
age_plot = alt.Chart(player_data).mark_point(opacity=0.4).encode(
    x=alt.X('age:Q').title("Age"),
    y=alt.Y("played_hours:Q").title("Played Hours"),
)
age_plot

<<<<<<< local


alt.Chart(...)

alt.Chart(...)

>>>>>>> remote


**Figure 2:** **Preliminary graph depicting outlier data from players.csv**. Scatter plot of quantitive variables from the Dataframe,`player_data`. `age`(x-axis) is plotted against `played_hours`(y-axis). **n=196** player data represented in graph.

In [ ]:
#Remove outlier. 4 datapoints with over 100hr playtime and 2 datapoints with age over 60
filtered_player_data = player_data[(player_data['age'] <= 60) & (player_data['played_hours'] <= 100)]

#Plot filter preliminary graph
age_plot_filtered = alt.Chart(filtered_player_data).mark_point(opacity=0.4).encode(
    x=alt.X('age:Q').title("Age").scale(zero=False),
    y=alt.Y("played_hours:Q").title("Played Hours"),
)
age_plot_filtered

<<<<<<< local


alt.Chart(...)

alt.Chart(...)

>>>>>>> remote


**Figure 3:** **Low play time across all age range**. Scatter plot of quantitive variables `filtered_player_data`. `age`(x-axis) is plotted against `played_hours`(y-axis). **n=190** player data represented in graph.

In [ ]:
#Mean, Median, Max and Min of filtered_player_data
filtered_player_data_info= filtered_player_data['played_hours'].agg(['mean', 'median', 'max', 'min', 'std'])
print("Info:",filtered_player_data_info)

#Mode of filtered_player_data
filtered_player_data_mode=filtered_player_data['played_hours'].mode()
print("Mode",filtered_player_data_mode)

Info: mean       1.979474
median     0.100000
max       56.100000
min        0.000000
std        7.685742
Name: played_hours, dtype: float64
Mode 0    0.0
Name: played_hours, dtype: float64


#### KNN Regression ####

In [ ]:
#Split data into training and testing dataframes
player_training, player_testing = train_test_split(
    filtered_player_data,
    test_size=0.25,
    random_state=33,  
)
X_train = player_training[['age']] 
y_train = player_training['played_hours'] 

X_test = player_testing[['age']]  
y_test = player_testing['played_hours']

In [ ]:
# Preprocess the data, make the pipeline
age_pipe = make_pipeline(
    StandardScaler(),
    KNeighborsRegressor(),
)

In [ ]:
# create the 5-fold GridSearchCV object
param_grid = {
    'kneighborsregressor__n_neighbors': range(1, 111, 1) #neighbors ranging from 1 to 110
}
age_tuned = GridSearchCV(
    age_pipe, 
    param_grid,
    cv=5, 
    n_jobs=-1, 
    scoring='neg_root_mean_squared_error'
)

# Fit the GridSearchCV object and retrieve the CV scores
age_results = pd.DataFrame(age_tuned.fit(X_train, y_train).cv_results_) 
age_results

<<<<<<< local
/opt/conda/lib/python3.11/site-packages/numpy/ma/core.py:2846: RuntimeWarning: invalid value encountered in cast
  _data = np.array(data, dtype=dtype, copy=copy,
>>>>>>> remote
<<<<<<< local <modified: text/html, text/plain>


,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_kneighborsregressor__n_neighbors,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.003164,0.000374,0.005336,0.005973,1,{'kneighborsregressor__n_neighbors': 1},-0.722066,-12.073896,-11.456517,-6.681023,-9.002519,-7.987204,4.105031,107
1,0.002834,0.000022,0.001950,0.000021,2,{'kneighborsregressor__n_neighbors': 2},-7.639823,-15.316804,-11.227851,-6.806057,-6.596516,-9.517410,3.345653,110
2,0.002796,0.000010,0.001955,0.000042,3,{'kneighborsregressor__n_neighbors': 3},-5.155806,-13.604928,-11.103494,-6.162283,-6.629261,-8.531154,3.255206,109
3,0.002783,0.000010,0.001911,0.000005,4,{'kneighborsregressor__n_neighbors': 4},-4.238870,-13.077508,-11.234713,-5.857979,-5.912204,-8.064255,3.444331,108
4,0.002770,0.000010,0.001921,0.000017,5,{'kneighborsregressor__n_neighbors': 5},-3.410022,-12.747023,-11.284856,-6.075152,-5.287915,-7.760994,3.610187,106
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
105,0.002716,0.000023,0.002032,0.000011,106,{'kneighborsregressor__n_neighbors': 106},-2.242995,-12.079505,-10.663243,-5.657835,-4.618947,-7.052505,3.723086,68
106,0.002703,0.000008,0.002025,0.000008,107,{'kneighborsregressor__n_neighbors': 107},-2.300962,-12.056163,-10.660851,-5.665565,-4.619813,-7.060671,3.700680,73
107,0.002705,0.000009,0.002031,0.000012,108,{'kneighborsregressor__n_neighbors': 108},-2.299375,-12.061981,-10.657457,-5.650980,-4.617129,-7.057384,3.703457,72
108,0.002705,0.000007,0.002044,0.000020,109,{'kneighborsregressor__n_neighbors': 109},-2.284726,-12.049145,-10.665866,-5.653414,-4.616823,-7.053995,3.705253,70


,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_kneighborsregressor__n_neighbors,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.005486,0.000254,0.003781,0.000244,1,{'kneighborsregressor__n_neighbors': 1},-0.722066,-12.073896,-11.456517,-6.681023,-9.002519,-7.987204,4.105031,107
1,0.007697,0.004346,0.003553,0.000046,2,{'kneighborsregressor__n_neighbors': 2},-7.639823,-15.316804,-11.227851,-6.806057,-6.596516,-9.517410,3.345653,110
2,0.005230,0.000040,0.003541,0.000081,3,{'kneighborsregressor__n_neighbors': 3},-5.155806,-13.604928,-11.103494,-6.162283,-6.629261,-8.531154,3.255206,109
3,0.005307,0.000232,0.003556,0.000086,4,{'kneighborsregressor__n_neighbors': 4},-4.238870,-13.077508,-11.234713,-5.857979,-5.912204,-8.064255,3.444331,108
4,0.005203,0.000069,0.003512,0.000067,5,{'kneighborsregressor__n_neighbors': 5},-3.410022,-12.747023,-11.284856,-6.075152,-5.287915,-7.760994,3.610187,106
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
105,0.004993,0.000007,0.003701,0.000031,106,{'kneighborsregressor__n_neighbors': 106},-2.242995,-12.079505,-10.663243,-5.657835,-4.618947,-7.052505,3.723086,68
106,0.004997,0.000011,0.003702,0.000015,107,{'kneighborsregressor__n_neighbors': 107},-2.300962,-12.056163,-10.660851,-5.665565,-4.619813,-7.060671,3.700680,73
107,0.004991,0.000033,0.003693,0.000028,108,{'kneighborsregressor__n_neighbors': 108},-2.299375,-12.061981,-10.657457,-5.650980,-4.617129,-7.057384,3.703457,72
108,0.005008,0.000029,0.003717,0.000028,109,{'kneighborsregressor__n_neighbors': 109},-2.284726,-12.049145,-10.665866,-5.653414,-4.616823,-7.053995,3.705253,70


>>>>>>> remote <modified: text/html, text/plain>


In [ ]:
#Find the best K and its RMSPE value
age_min = age_tuned.best_params_
age_best_RMSPE = -age_tuned.best_score_
print("Best Parameters (age_min):", age_min)
print("Best RMSPE (age_best_RMSPE):", age_best_RMSPE)

Best Parameters (age_min): {'kneighborsregressor__n_neighbors': 70}
Best RMSPE (age_best_RMSPE): 6.946063563917676


In [ ]:
#Evaluating RMSPE on the test set
age_prediction = age_tuned.predict(X_test)
age_summary = mean_squared_error(
    y_true=y_test, 
    y_pred=age_prediction
)**(1/2)
print("RMPSE of test set:", age_summary)

RMPSE of test set: 6.9597827719687615


In [ ]:
np.random.seed(33)
#Predict the hours played for age
age_preds = filtered_player_data.assign(
    predictions= age_tuned.predict(filtered_player_data[['age']])
)
#Plot all players
age_plot = alt.Chart(age_preds).mark_circle(opacity=0.4).encode(
    x=alt.X('age').title('Age').scale(zero=False),
    y=alt.Y('played_hours').title('Hours Played')
)
#Add prediction line
age_plot = age_plot + alt.Chart(age_preds, title= "K=70").mark_line(color="Black").encode(
    x="age",
    y="predictions",
)
age_plot

<<<<<<< local


alt.LayerChart(...)

alt.LayerChart(...)

>>>>>>> remote


**Figure 3:** **No relationship between age and hours played.** Scatter plot of quantitive variables `filtered_player_data` with a prediction line(Black). `age`(x-axis) is plotted against `played_hours`(y-axis). **n=190** player data represented in graph. Predicted values of hours played (black line) for K-NN regression model (K=70).

### Experience ###

#### Preliminary Graph ####

In [20]:
#Plot preliminary graph
experience_plot = alt.Chart(player_data).mark_point(opacity=0.4).encode(
    x=alt.X('experience:N').title("Experience"),
    y=alt.Y("played_hours:Q").title("Played Hours"),
)
experience_plot

alt.Chart(...)

#### Remove outlier ####
To assess meaningful patterns, the playtime range was adjusted to 0-5 hours.

In [50]:
# Remove outliers for played_hours
filtered_player_data = player_data[player_data['played_hours'] <= 5]

#Plot filter preliminary graph
experience_plot_filtered = alt.Chart(filtered_player_data).mark_point(opacity=0.4).encode(
    x=alt.X('experience:N').title("Experience").scale(zero=False),
    y=alt.Y("played_hours:Q").title("Played Hours"),
)
experience_plot_filtered

alt.Chart(...)

### KNN Regresion ###

In [54]:
#change experience value to numeric value
filtered_player_data['experience_numeric'] = filtered_player_data['experience'].replace({
    'Amateur': 1, 
    'Beginner': 2, 
    'Pro': 3, 
    'Regular': 4, 
    'Veteran': 5
})

X = filtered_player_data[['experience_numeric']]
y = filtered_player_data['played_hours']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=33)

/tmp/ipykernel_376/626172549.py:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  filtered_player_data['experience_numeric'] = filtered_player_data['experience'].replace({
/tmp/ipykernel_376/626172549.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_player_data['experience_numeric'] = filtered_player_data['experience'].replace({


In [55]:
experience_param_grid = {'n_neighbors': range(1, 21)} 

grid_search = GridSearchCV(
    KNeighborsRegressor(), 
    experience_param_grid, 
    cv=5, 
    scoring='neg_mean_squared_error')
grid_search.fit(X_train, y_train)

best_k = grid_search.best_params_['n_neighbors']
best_mse = -grid_search.best_score_

print(best_k)
print(best_mse)

15
0.5971404115226336


In [60]:
knn_regressor = KNeighborsRegressor(n_neighbors=best_k)
knn_regressor.fit(X_train, y_train)

new_experience = pd.DataFrame({'experience_numeric': [1]})
predicted_hours = knn_regressor.predict(new_experience)

### Classification using KNN classfier 

First, we choose variables that will be the predictors. hashedEmail, name, individualID, and organizationName are unique identifiers for the observation. The variables are not related to any measured property of the cells, and should not be used as predictors. We choose the rest of the variables using forward selection and we know that there are no missing values in our predictors from previous EDA. Played_hours is categorized and we use play_time to replace played_hours.

In [128]:
# choose 20% of data to be test data
train_df, test_df = train_test_split(player_data, test_size=0.20, random_state=123)

X_train, y_train = (
    train_df.drop(columns=["hashedEmail", "name", "individualId", "organizationName", "play_time"]),
    train_df["play_time"],
)
X_test, y_test = (
    test_df.drop(columns=["hashedEmail", "name", "individualId", "organizationName", "play_time"]),
    test_df["play_time"],
)


variables = ["experience", "subscribe", "gender", "age"]

accuracy_dict = {"size": [], "selected_predictors": [], "accuracy": []}

# store the total number of predictors
n_total = len(variables)

# start with an empty list of selected predictors
selected = []

# create the pipeline and CV grid search objects
param_grid = {
    "kneighborsclassifier__n_neighbors": range(1, 15, 1),
}

numeric_features = ["age"]
# "experience" is ordinal because we assume there is a ordering in which more experienced players spend more time to play
ordinal_features = ["experience", "subscribe"]
ordering= [["Beginner", "Amateur", "Regular", "Pro", "Veteran"], [False, True]]
categorical_features = ["gender"]

play_preprocessor = make_column_transformer(
    (StandardScaler(), numeric_features),
    (OrdinalEncoder(categories=ordering), ordinal_features),
    (OneHotEncoder(handle_unknown="ignore", sparse_output=False), categorical_features),
    remainder="passthrough",
    verbose_feature_names_out=False
)

player_tune_pipe = make_pipeline(play_preprocessor, KNeighborsClassifier())
player_tune_grid = GridSearchCV(
    estimator=player_tune_pipe,
    param_grid=param_grid,
    cv=10,
    n_jobs=-1
)
player_tune_grid.fit(X_train, y_train)
accuracies_grid = pd.DataFrame(player_tune_grid.cv_results_)
accuracies_grid["sem_test_score"] = accuracies_grid["std_test_score"] / 10**(1/2)
accuracies_grid = (
    accuracies_grid[[
        "param_kneighborsclassifier__n_neighbors",
        "mean_test_score",
        "sem_test_score"
    ]]
    .rename(columns={"param_kneighborsclassifier__n_neighbors": "n_neighbors"})
)
print(player_tune_grid.best_params_)
accuracies_grid

{'kneighborsclassifier__n_neighbors': 4}


,n_neighbors,mean_test_score,sem_test_score
0,1,0.974167,0.010014
1,2,0.974583,0.009850
2,3,0.980417,0.009466
3,4,0.986667,0.008433
4,5,0.980417,0.009466
5,6,0.986667,0.008433
6,7,0.967917,0.010156
7,8,0.967917,0.010156
8,9,0.961667,0.009909
9,10,0.961667,0.009909


In [22]:
accuracy_vs_k = alt.Chart(accuracies_grid).mark_line(point=True).encode(
    x=alt.X("n_neighbors").title("Neighbors"),
    y=alt.Y("mean_test_score")
        .scale(zero=False)
        .title("Accuracy estimate")
)

accuracy_vs_k

alt.Chart(...)

The highest accuracy is achieved at k=4, which is consistent with the plot where the score is highest at 4. This point is reasonable because it's not too high which causes underfitting, nor it's too low which causes overfitting. The score is high. We use the chosen hyperparameter to predict on test set and see if the test score is low and it's overfitting.

In [23]:
test_df["predicted"] = player_tune_grid.predict(X_test)
player_tune_grid.score(X_test, y_test)

0.975

In [124]:
precision_score(test_df["play_time"], test_df["predicted"], pos_label="high")

1.0

In [121]:
recall_score(test_df["play_time"], test_df["predicted"], pos_label="high")

0.8

In [125]:
pd.crosstab(test_df["play_time"], test_df["predicted"])

predicted,high,low
play_time,,
high,4,1
low,0,35


We want to identify the high play_time in the task so we are setting high to be the positive class in calculating the scores.The score overall is high. The accuracy is high so the correctly identified test observations were a large portion of the total observations. Also, precision is high. Precision quantifies how many of the positive predictions the classifier made were actually positive and our precision is 1 so the false positives were 0. The recall is lower and we have 1 false negative. The scores do not give a totally accurate representation because the dataset size is small and we do not have a lot of test observations to test our model on.

## Discussion ##

### Age ###

Using KNNregression, analysis indicated that there is no relationship between age and PLAICraft playtime, as shown by the prediction line being horizontal. The RMSPE value for the model was 6.95 hours, suggesting that any given prediction could vary by 6.95 hours. This closely matches the test set’s RMPSE of 6.96 hours, suggesting the model generalizes well to new data. However, the RMSPE value is not within an acceptable range. It is too large compared to the observed playtime values, which have an average of 1.98 hours and mode of 0.0 hours (in the filtered dataset). This is likely a result of high variability in the data. Although the model seems to perform consistently the predictions will not be useful due to its large error margins. 
Although the model is unreliable, the data was still surprising. It was expected the data would show a parabolic relationship between playtime and age, with playtime peaking among young adults (18-29 years old) and then decreasing with increasing age. This expectation comes from the literature, which indicates that 67% of young adults are gamers, compared to 40% of older adults (50-64 years old) and 25% of seniors (65+) (Bunz et al., 2020).

## References ##

* Bunz U.,  Cortese J., Sellers N. (2020). Examining younger and older adults' digital gaming habits and health measures. Gerontechnology, 19:4. https://doi.org/10.4017/gt.2020.19.04.381